### 제품 정보 페이지 크롤링 함수

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import json
import datetime
from seleniumbase import SB
from bs4 import BeautifulSoup


def get_product_detail_info(sb, goods_no: str) -> dict:
    url = f"https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo={goods_no}"
    sb.uc_open_with_reconnect(url, reconnect_time=5)
    time.sleep(1)
    html = sb.driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 카테고리 추출
    try:
        category = soup.select_one("a.cate_y#midCatNm").text.strip()
    except Exception:
        category = ""

    # 총리뷰수
    try:
        review_info = soup.select_one("#repReview em")
        total_review = int(review_info.text.strip().replace("(", "").replace("건)", "").replace(",", ""))
    except Exception as e:
        print(f"총 리뷰수 파싱 실패: {e}")
        total_review = 0

    # 리뷰평점
    try:
        review_score = soup.select_one("#repReview b")
        review_score = float(review_score.text.strip())
    except Exception as e:
        print(f"리뷰평점 파싱 실패: {e}")
        review_score = None

    # 리뷰 분포 기본값
    pctOf5 = pctOf4 = pctOf3 = pctOf2 = pctOf1 = None
    review_detail = ""

    # 리뷰가 1건 이상 있을 때만 리뷰탭 클릭 및 분포 수집
    if total_review > 0:
        try:
            sb.click("a.goods_reputation")
            WebDriverWait(sb.driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "ul.graph_list span.per"))
            )
            percent_elements = sb.find_elements("css selector", "ul.graph_list span.per")
            percent_list = [el.text.strip() for el in percent_elements]
            if len(percent_list) == 5:
                pctOf5 = percent_list[0]
                pctOf4 = percent_list[1]
                pctOf3 = percent_list[2]
                pctOf2 = percent_list[3]
                pctOf1 = percent_list[4]

            # reviewDetail 정보
            review_detail = []
            polls = sb.find_elements("css selector", "dl.poll_type2.type3")
            for poll in polls:
                try:
                    title = poll.find_element("css selector", "span").text.strip()
                    li_tags = poll.find_elements("css selector", "ul.list > li")
                    for li in li_tags:
                        label = li.find_element("css selector", "span.txt").text.strip()
                        percent = li.find_element("css selector", "em.per").text.strip()
                        review_detail.append({
                            "type": title,
                            "value": label,
                            "gauge": percent
                        })
                except Exception as e:
                    print(f"리뷰 설문 수집 오류: {e}")
            review_detail = json.dumps(review_detail, ensure_ascii=False)

        except Exception as e:
            print("리뷰 정보 없음:", e)

    # === 상세스펙(구매정보) 추출 ===
    # 구매정보 탭 클릭
    try:
        sb.click("a.goods_buyinfo")
        time.sleep(1)  # ajax 로딩 대기
        html = sb.driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
    except Exception as e:
        print("구매정보 탭 클릭 실패:", e)

    # === 한글 키 → 영어 키 매핑 ===
    title_map = {
        "용량": "capacity",
        "주요 사양": "detail",
        "모든 성분": "ingredient"
    }

    # 기본값 세팅
    detail_spec = {
        "capacity": "",
        "detail": "",
        "ingredient": ""
    }

    try:
        dl_tags = soup.select("div#artcInfo dl.detail_info_list")
        for dl in dl_tags:
            dt = dl.select_one("dt")
            dd = dl.select_one("dd")
            if dt and dd:
                dt_text = dt.text.strip()
                dd_text = dd.text.strip()

                for kr_title, en_key in title_map.items():
                    if kr_title in dt_text:
                        detail_spec[en_key] = dd_text
    except Exception as e:
        print(f"[상세 스펙 파싱 오류]: {e}")

    return {
        "category": category,
        "numOfReviews": total_review,
        "avgReview": review_score,
        "pctOf5": pctOf5,
        "pctOf4": pctOf4,
        "pctOf3": pctOf3,
        "pctOf2": pctOf2,
        "pctOf1": pctOf1,
        "reviewDetail": review_detail,
        **detail_spec,
    }

### 브랜드관 페이지 크롤링 함수

In [17]:
from seleniumbase import SB
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import time


def get_brand(brand_code) -> pd.DataFrame:
    url = f"https://www.oliveyoung.co.kr/store/display/getBrandShopDetail.do?onlBrndCd={brand_code}"
    data = []
    collected_at = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    with SB(uc=True, test=True, headless=True) as sb:
        sb.uc_open_with_reconnect(url, reconnect_time=20)
        time.sleep(2)

        page = 1
        while True:
            if page > 1:
                try:
                    # 페이지네이션 버튼 클릭 (페이지가 없으면 break)
                    sb.click(f"div.pageing a[data-page-no='{page}']")
                    time.sleep(2)  # ajax 로딩 대기
                except Exception as e:
                    print(f"{page}페이지 버튼 클릭 실패 또는 더 이상 페이지 없음: {e}")
                    break

            html = sb.driver.page_source
            soup = BeautifulSoup(html, "html.parser")

            # 브랜드명 추출
            try:
                brand = soup.select_one("h2.title-detail-brand").text.strip()
            except Exception:
                brand = "식물나라"

            # 상품 목록 추출
            items = soup.select("ul.prod-list.goodsProd > li")
            if not items:
                print(f"{page}페이지에 상품이 없습니다.")
                break

            for item in items:
                is_pb = 1
                try:
                    name = item.select_one("span.prod-name.double-line").text.strip()
                except Exception:
                    name = ""
                try:
                    a_tag = item.select_one("a[data-ref-goodsno]")
                    goods_no = a_tag["data-ref-goodsno"] if a_tag else ""
                except Exception:
                    goods_no = ""
                try:
                    price_final = item.select_one("strong.total").text.strip().replace("원", "").replace(",", "").replace("~", "")
                except Exception:
                    price_final = ""
                try:
                    price_original = item.select_one("span.origin").text.strip().replace("원", "").replace(",", "")
                except Exception:
                    price_original = ""
                try:
                    flag_spans = item.select("div.flags span.flag")
                    flag_list = [span.text.strip() for span in flag_spans if span.text.strip()]
                    flag_str = ",".join(flag_list) if flag_list else ""
                except Exception:
                    flag_str = ""
                try:
                    soldout_flag = item.select_one("span.status_flag.soldout")
                    is_soldout = bool(soldout_flag)
                except Exception:
                    is_soldout = False

                data.append({
                    "brandName": brand,
                    "isPB": is_pb,
                    "goodsName": name,
                    "goodsNo": goods_no,
                    "salePrice": price_final,
                    "originalPrice": price_original,
                    "flagList": flag_str,
                    "isSoldout": is_soldout,
                    "createdAt": collected_at
                })
            # 다음 페이지로
            page += 1

    return pd.DataFrame(data)


### 실행 코드

In [18]:
#### 실행 코드 #####
PB_BRAND_CODE_DICT = {
    "바이오힐 보": "A000897",
    "브링그린": "A002253",
    "웨이크메이크": "A001240",
    "컬러그램": "A002712",
    "필리밀리": "A002502",
    "아이디얼포맨": "A001643",
    "라운드어라운드": "A001306",
    "식물나라": "A000036",
    "케어플러스": "A003339",
    "탄탄": "A015673",
    "딜라이트 프로젝트": "A003361",
}

# for brand_name, brand_code in PB_BRAND_CODE_DICT.items():
df = get_brand("A000036")

with SB(uc=True, test=True) as sb:
    detail_list = []
    for goods_no in df['goodsNo']:
        detail = get_product_detail_info(sb, goods_no)
        detail_list.append(detail)

==================== {  File "<frozen runpy>:198:SB} starts ====================
4페이지 버튼 클릭 실패 또는 더 이상 페이지 없음: Message: 
 Element {div.pageing a[data-page-no='4']} was not present after 7 seconds!

=============== {  File "<frozen runpy>:198:SB} passed in 51.19s ===============
==================== {  File "<frozen runpy>:198:SB} starts ====================
============== {  File "<frozen runpy>:198:SB} passed in 1242.44s ==============


In [19]:
detail_df = pd.DataFrame(detail_list)
result_df = pd.concat([df.reset_index(drop=True), detail_df.reset_index(drop=True)], axis=1)

now_str = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
result_df.to_json(f'suncare_result_{now_str}.json', orient='records', force_ascii=False, indent=2)

In [20]:
result_df

,brandName,isPB,goodsName,goodsNo,salePrice,originalPrice,flagList,isSoldout,createdAt,category,...,avgReview,pctOf5,pctOf4,pctOf3,pctOf2,pctOf1,reviewDetail,capacity,detail,ingredient
0,식물나라,1,식물나라 워터프루프/알로에 쿨링 선 스프레이 2종 (단품/1+1),A000000212555,8800,14800,세일,False,2025-07-06 18:44:34,선케어,...,4.7,81%,13%,5%,1%,1%,[],100ML / 100ML+100ML,모든 피부 타입,"[워터프루프 선 스프레이]정제수,다이실록세인,사이클로펜타실록세인,에탄올,호모살레이트..."
1,식물나라,1,[7월 올영픽/NEW] 식물나라 시원한 데오티슈 15매 산리오 에디션 (무향 쿠로미),A000000228882,3600,4500,"1+1,쿠폰",False,2025-07-06 18:44:34,바디케어,...,4.8,84%,10%,4%,1%,1%,"[{""type"": ""지속력"", ""value"": ""지속이 오래돼요"", ""gauge"":...",15매,■ 모든 피부용,"[데오티슈 무향]정제수, 에탄올, 빙하수, 소듐벤조에이트, 피이지-60하이드로제네이..."
2,식물나라,1,[대용량] 식물나라 저자극 페이스 앤 바디 선 크림 150ml,A000000221882,10800,19800,"세일,쿠폰",False,2025-07-06 18:44:34,선케어,...,4.7,79%,15%,5%,1%,1%,"[{""type"": ""피부타입"", ""value"": ""건성에 좋아요"", ""gauge"":...",150mL,■ 모든 피부용,"정제수, 징크옥사이드 (CI 77947), 부틸옥틸살리실레이트, 아이소프로필미리스테..."
3,식물나라,1,식물나라 보송 페이스 앤 바디 선 스틱 30g (단품/1+1),A000000150460,13800,27000,세일,False,2025-07-06 18:44:34,선케어,...,4.8,83%,12%,4%,0%,1%,[],■ 30g,■ 모든 피부용,"부틸렌글라이콜다이카프릴레이트/다이카프레이트, 폴리에틸렌, 다이메티콘, 옥토크릴렌, ..."
4,식물나라,1,식물나라 보송 페이스 앤 바디 선 스틱 30g 1+1 기획,A000000228157,18800,27000,세일,False,2025-07-06 18:44:34,선케어,...,4.8,83%,12%,4%,0%,1%,[],■ 30g+30g,■ 모든 피부용,"부틸렌글라이콜다이카프릴레이트/다이카프레이트, 폴리에틸렌, 다이메티콘, 옥토크릴렌, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,식물나라,1,식물나라 제주 탄산수 딥 모공 클렌징폼150mL[단품/ 1+1기획],A000000184597,12800,,,True,2025-07-06 18:44:34,클렌징,...,4.8,85%,11%,3%,0%,0%,"[{""type"": ""피부타입"", ""value"": ""건성에 좋아요"", ""gauge"":...",■ 본품150mL,■ 모든피부용,"정제수(탄산수), 미리스틱애씨드, 글리세린, 메틸프로판다이올, 포타슘하이드록사이드,..."
64,식물나라,1,식물나라 호두 보들 스크럽 바디워시 200mL,A000000191753,11800,,,True,2025-07-06 18:44:34,바디케어,...,4.6,76%,16%,6%,2%,1%,"[{""type"": ""세정력"", ""value"": ""아주 만족해요"", ""gauge"": ...",200mL,■ 모든 피부용,"정제수,소듐C14-16올레핀설포네이트,글리세린,아크릴레이트코폴리머,라우릴베타인,마이..."
65,식물나라,1,식물나라 유채꿀 촉촉 멀티오일30mL,A000000199400,10900,17800,세일,True,2025-07-06 18:44:34,스킨케어,...,4.7,82%,10%,5%,2%,1%,"[{""type"": ""피부타입"", ""value"": ""건성에 좋아요"", ""gauge"":...",30mL,■ 모든 피부용,"정제수, 펜틸렌글라이콜, 글리세린, 메틸글루세스-20, 에탄올, 나이아신아마이드, ..."
66,식물나라,1,식물나라 제주 탄산수 딥 립 앤 아이 리무버 150mL(24AD),A000000204481,8900,,,True,2025-07-06 18:44:34,클렌징,...,4.8,85%,11%,3%,0%,0%,"[{""type"": ""피부타입"", ""value"": ""건성에 좋아요"", ""gauge"":...",150mL,■ 모든 피부용,"정제수,사이클로펜타실록세인,카프릴릭/카프릭트라이글리세라이드,다이프로필렌글라이콜,아이..."
